# Discrete Fourier Transform

## What is the fourier transform?
Every signal can be decomposed into a superposition of sinusoids. (A summed combination of shifted and scaled sinusoids).

The **Discrete Fourier Transform (DFT)** is just the discrete version of this idea.

## Example: Support Volume

In [ ]:
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import sys
sys.path.append('../')
sys.path.append('../musimathics')

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc, patches
import matplotlib.cm as cm
import matplotlib.ticker as mtick

from ipython_animation import create_animation, DEFAULT_FPS

date_parser = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S+00')
df = pd.read_csv('helpscout_conversations.csv', header=None, usecols=[0,10], names=['id', 'created_at'], date_parser=date_parser, index_col=['created_at'])
df.sort_index(inplace=True)
support_count_per_day = df.resample('D').apply('count').rename(columns={'id':'count'})
# interpolate outliers to 1/2 between neighbors
outlier_index = np.argwhere(support_count_per_day['count'].values > 2000)[0][0]
support_count_per_day.iloc[outlier_index]['count'] = (support_count_per_day.iloc[outlier_index - 1]['count'] + support_count_per_day.iloc[outlier_index + 1]['count']) / 2

In [ ]:
%matplotlib inline
_ = np.seterr(divide='ignore')

In [ ]:
# verify that the error is effectively 0 (we've exactly reproduced original time-domain signal from its frequency components!)
def rmse(estimated, actual):
    return np.sqrt(((estimated - actual) ** 2).sum() / actual.size)

def formatted_rmse(estimated, actual):
    return 'RMSE: {0:.3f}'.format(rmse(estimated, actual))

In [ ]:
num_days = 30
num_sine_components = 16

support_counts = support_count_per_day[-num_days:]

x = support_counts['count'].values
N = len(x)
hN = N // 2 + 1
X = np.fft.rfft(x)
ordered_X_indices = np.abs(X).argsort()[::-1]
X_max = 2 * np.abs(X[1:]).max() / N
k = np.arange(N)
sample_rate_days = 1
frequencies = ((sample_rate_days * k) / N)[:hN]
period = (1 / frequencies)[1:]

fig, axes = plt.subplots(3, 1, figsize=(12, 14))
reconstruction_plot, sine_plot, X_plot = axes

reconstruction_plot.set_title('Support Volume (per-day)')
reconstruction_plot.set_xlabel('created_at day')
reconstruction_plot.set_ylabel('Ticket count')
actual_line, = reconstruction_plot.plot(support_counts.index, x)
n_point_idft_line, = reconstruction_plot.plot(support_counts.index, x)
nth_sinusoid_line, = sine_plot.plot(np.zeros(N))
rmse_patch = patches.Patch(color='red')

fig.suptitle('Reconstructing a signal with the DFT', size=16)

sine_plot.set_title('nth Sinusoid')
sine_plot.set_ylabel('Amplitude')

period_axis = X_plot.twiny()
X_plot.set_title('Spectrum')
X_plot.set_xlabel('Frequency (1/days)')
X_plot.set_ylabel('Magnitude')

period_axis.set_xlabel('Period (days)')
period_label_indices = np.round(np.linspace(0, len(period) - 1, num_sine_components)).astype(int)
period_axis.set_xticks(np.arange(hN)[period_label_indices])
period_axis.set_xlim(-1, len(period) - 1)
period_axis.set_xticklabels('%0.2f' % p for p in period[period_label_indices])

reconstruction_plot.set_xlim(support_counts.index.min(), support_counts.index.max())
X_plot.set_xlim(frequencies[0], frequencies[-1])
X_plot.set_ylim(0, np.abs(X[1:]).max() * 2.1 / N)
sine_plot.set_xlim(0, N - 1)
sine_plot.set_ylim(-X_max, X_max)

plt.tight_layout()
plt.subplots_adjust(top=0.92)

reconstruction = np.zeros(N) # cumulatively add component sinusoids
skip = True
def animate_reconstruction(i):
    global skip
    global reconstruction
    if skip: # frame 0 animates twice, so annoying
        skip = False
        return

    X_index = ordered_X_indices[i]
    X_value = X[X_index]
    ith_sinusoid = np.abs(X_value) * np.cos(2 * np.pi * X_index * np.arange(0, N) / N + np.angle(X_value)) / N
    if X_index != 0 and X_index != hN - 1: 
        # DFT is Hermitian-symmetric.
        # We only count it twice if it's not the DC offset or the middle
        ith_sinusoid *= 2

    reconstruction += ith_sinusoid
    n_point_idft_line.set_ydata(reconstruction)
    nth_sinusoid_line.set_ydata(ith_sinusoid)

    X_plot.vlines(x=frequencies[ordered_X_indices[:i+1]], ymin=0, ymax=np.abs(X[ordered_X_indices[:i+1]]) * 2 / N)

    sine_plot.set_title('Sinusoid Component #%i' % (i + 1))
    reconstruction_plot.legend(handles=[actual_line, n_point_idft_line, rmse_patch], \
               labels=['Actual', '{0:d}-point IDFT'.format(i + 1), formatted_rmse(reconstruction, x)], loc=3)

In [ ]:
create_animation(fig, plt, animate_reconstruction, frames=num_sine_components, frames_per_second=1)

## What assumptions does this model make?

* The given signal can be decomposed into a sum of sinusoids with different phases and amplitudes
* The given signal is exactly one period of an infinitely repeating periodic signal
* The frequencies of the sinusoids making up the signal are exact multiples of the fundamental (lowest frequency)
    - All energy is constrained to discrete "bins", so "true" frequencies between those bins get spread across nearby frequencies
* The highest frequency in the data being sampled is at most $\frac{\text{sample_rate}}{2}$ (Nyquist frequency)

## What is it good for?

### Some applications
* Image processing & compression (e.g. JPEG)
* Audio/signal compression (e.g. MP3)
* Convolution
* Multiplication of polynomials and large integers
* **Audio analysis/synthesis** _(audio is also at the root of the discovery of the DFT!)_

# History

(Main source: [Julius O. Smith's History of Virtual Musical Instruments based on Physical Modelling](https://www.youtube.com/watch?v=I64y40EIPaM&t=1362s))

## Strings

### Martin Mersenne (1636)
> [Since the vibrating string] produces five or six tones..., it seems that it is entirely necessary that it beat the air five, four, three and two times at the same time, which is impossible to imagine unless one says that half of the string beats the air twice, one third beats it three times, etc. while the whole string beats it once. This picture runs against experience, which clearly shows that all parts of the string make the same number of returns in the same time, because the continuous string has a single motion, even though parts near the bridge move more slowly.

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('Qr_rxqwc1jE')

### Joseph Sauveur (1701)
> While meditating on the phenomena of sound, I was made to observe that especially at night one may hear from long strings not only the principal sound, but also other small sounds, a twelfth and a seventeenth above... I concluded that the string in addition to the undulations it makes in its entire length so as to form the fundamental sound may divide itself in two, three, four, etc. undulations which form the octave, the twelfth, the fifteenth of this sound.

Sauveur plucked a monochord having a light obstacle mounted to create "nodes", and was surprised that the string did not move at the nodal points.

![](https://i.stack.imgur.com/pA0iS.jpg)

Sauveur coined the term _**node**_, and the term _**harmonic**_, so named because they are "harmonious" with the fundamental.

### Brook Taylor (1713)
* First to derive the string fundamental frequency,  
  $f = \frac{\sqrt{K / \epsilon}}{2L}$,  
  where $K$ = string tension, $\epsilon$ = mass density, $L$ = string length
* Derived that the string restoring force is proportional to the string _curvature_
* (Wrongly) concluded that even though many _initial_ shapes were clearly possible, the vibration would quickly transition to a sinusoidal shape (only the funcamental mode vibration was supported)

### Jean-Phillippe Rameau (1726)

* Rameau was a composer interested in studying harmonic overtones to establish a more scientific foundation for music theory
* Anticipated a kind of spectrum analyzer theory of hearing:
> What has been said of [the separate vibrating modes of] sonorous bodies should be applied equally to the fibers which carpet the bottom of the ear's cochlea; these fibers are so many sonorous bodies, to which the air transmits its vibrations, and from which the perception of sounds and harmony is carried to the soul
Thus, Rameau regarded the hair cells as a bank of little sympathetic resonators.

### Daniel Bernoulli (1733)

* Studied the vertically suspended chain
* Observed multiple modes of vibration (as many as there were masses), and their inharmonicity
* Realized that the infinitely long chain was equivalent to an infinitely long musical string
* Later studied elastic bands
> Both sounds exist at once and are very distinctly perceived. ... This is no wonder, since neither oscillation helps or hinders the other; indeed, when the band is curved by reason of one oscillation, it may always be considered as straight in respect to another oscillation, since the oscillations are virtually infinitely small. Therefore oscillations of any kind may occur, whether the band be destitute of all other oscillation or executing others at the same time. In free bands, ... I have often perceived three or four sounds at the same time.
* _**Superposition**_ of small vibrations at different frequencies clearly conceived

### Jean La Rond D'Alembert (1746)

* invented the partial differential equation (PDE) by plugging Taylor's resoring force $f$ into Newton's $f = ma$ writting as a _differential form_ (as developed by Euler)
* Showed that any solution was a _traveling wave_ to left and/or right
* Showed that Taylor's sinusoidal fundamental mode was a special case solution
* Disagreed with Taylor that all initial conditions lead to a single sinusoid
* Did not allow an initial triangular shape (not twice differentiable), and suggested using a beaded string for this case

### Leonard Euler (1749)

* Quickly generalized d'Alembert's results to any initial string shape
* Showed that the solution space included sums of sines:  
$y(x,t) = \sum\limits_{k}{A_k\sin(k\pi x/L)\cos(k \pi f_0 t})$  
(displacement of length $L$ vibrating string at time $t$, position $x$)  
[This gets us a long way!](https://karlhiner.com/processing/string_pluck)
* He did not consider this a general solution (one "obviously" couldn't make a triangular initial shape out of sines)
* Fourier wasn't born yet!

In [ ]:
def nth_harmonic(x, n):
    fft = np.fft.fft(x)
    fft[n + 1:] = 0
    return np.fft.ifft(fft).real

N = 100
def animate_harmonics(i):
    position = np.clip(i, 1, N - 1)
    x = np.concatenate([np.linspace(0, 1, position, False), np.linspace(1, 0, N - position)])
    x = np.concatenate([x, -x[::-1][1:]])
    K = 10
    harmonics = np.array([nth_harmonic(x, n) for n in range(K)]).T
    plt.cla()
    plt.plot(harmonics)
    plt.plot(x, label='Target "pluck" shape')
    plt.plot(2 * harmonics.sum(axis=1) / K, label='Approximation from sum of {%i} harmonics' % K)
    plt.xlabel('position along string')
    plt.ylabel('displacement')

In [ ]:
fig = plt.figure(figsize=(16, 5))
create_animation(fig, plt, animate_harmonics, frames=N - 2, default_mode='reflect')

In [ ]:
from mpl_toolkits.mplot3d.axes3d import Axes3D
# Bessel functions and their roots (zero crossings)
from scipy.special import jn, jn_zeros

# rows correspond to root number
# columns correspond to Bessel order
# (e.g. 2nd row, 1st column == 2nd root of 0-order Bessel function == "Mode 02")
fig = plt.figure(figsize=(12, 8))

bessel_order_max = 4
bessel_root_max = 3

axes = []
for x in range(bessel_order_max):
    axes.append([])
    for y in range(bessel_root_max):
        axes[x].append(fig.add_subplot(bessel_root_max, bessel_order_max, y * 4 + x + 1, projection='3d'))

dim = 50
r, phi = np.meshgrid(np.linspace(0, 1, dim), np.linspace(0, 2 * np.pi, dim))
x, y = r * np.cos(phi), r * np.sin(phi)

bessel_roots = np.array([jn_zeros(bessel_order, bessel_root_max) for bessel_order in range(bessel_order_max)])

jns = np.ndarray((bessel_order_max, bessel_root_max, dim, dim))
for m in range(bessel_order_max):
    for n in range(bessel_root_max):
        jns[m][n] = jn(m, bessel_roots[m, n] * r) * np.cos(m * phi)

num_frames = 20

def animate_drum_modes(i):
    t = 2 * np.pi * i / num_frames
    for m in range(bessel_order_max):
        for n in range(bessel_root_max):
            z = jns[m,n] * np.sin(t)
            ax = axes[m][n]
            ax.cla()
            plot = ax.plot_surface(x, y, z, cmap='jet', vmin=-1, vmax=1)
            ax.set_zlim(-1, 1)
            ax.set_title('Mode %i%i' % (m, n + 1))
            ax.set_xticks([], [])
            ax.set_yticks([], [])
            ax.set_zticks([], [])

In [ ]:
create_animation(fig, plt, animate_drum_modes, frames=num_frames, frames_per_second=DEFAULT_FPS / 2)

### Back to Daniel Bernoulli (1753)

Bernoulli was annoyed with d'Alembert and Euler

* He had published the superposition-of-sinusoids solution long ago
* The supposedly new solutions of d'Alembert and Euler were simply a mixture of simple modes
* He didn't like fusing the simple pure oscillations into a single formula
* Considered his "physical" analysis preferable to their abstract math

> I saw at once that one could admit this multitude of sine curves only in a sense altogether _improper_. I do not less admire the calculations of Messrs. d'Alembert and Euler, which certainly include what is most profound and most advanced in all of analysis, but which show at the same time that an abstract analysis, if heeded without any synthetic [physical] examination of the question proposed, is more likely to surprise than enlighten. It seems to me that giving attention to the nature of the vibrations of strings suffices to foresee without any calculation at all what these great geometers have found by the most difficult and abstract calculations that the analytic mind has yet conceived.

# The DFT as a model for audio

The DFT is an effective model for audio because the sources of most audio are combinations of driven harmonic oscillators.

# Physical Basis of Sound

## Longitudinal wave propagation

The following is a visualization (based on a [Mathematica notebook by Mats Bengtsson](http://library.wolfram.com/infocenter/MathSource/780/)) of the way sound waves propagate as longitudinal waves through a medium.  Notice that each particle only moves back and forth, but their collective movement creates an emergent wave with its own measurable characteristics.

The particle movements are the only thing being explicitly modelled here.  All other charts are derived from _empirical measurements_ of these virtual particles.  The units and magnitude aren't to be taken too seriously in this example.

In [ ]:
def displace_coordinates(coordinates, t):
    return np.maximum(0, coordinates + amplitude * np.sin(wave_constant * coordinates - angular_frequency * t))

In [ ]:
animation_length_seconds = 3
num_frames = DEFAULT_FPS * animation_length_seconds

num_atoms = 2400
tube_length = 100
tube_width = 20
wave_length = tube_length / 5
wave_constant = 2 * np.pi / wave_length
amplitude = 0.10 * wave_length
angular_frequency = 4 * np.pi

pressure_line_displacement = tube_length / 2

atom_coordinates = np.random.rand(num_atoms, 2) * (tube_length, tube_width)
atom_coordinates = atom_coordinates[atom_coordinates[:,0].argsort()]
total_pressure_values = np.zeros(num_frames)
old_coords = atom_coordinates.copy()
new_coords = atom_coordinates.copy()
diffs = np.zeros(old_coords.shape)

fig, axes = plt.subplots(5, 1, figsize=(12, 20))
fig.suptitle('Longitudinal Waves', size=16)

displacements_plot = axes[0]
density_plot = axes[1]
velocity_plot = axes[2]
total_velocity_plot = axes[3]
pressure_plot = axes[4]

displacements_plot.set_title('Particle displacements')
displacements_plot.set_xlabel('Displacement x (m)')
displacements_plot.set_ylabel('Displacement y (m)')
displacements_plot.set_xlim([0, tube_length])
displacements_plot.set_ylim([0, tube_width])

# highlight a few points on the scatter plot
highlight_points = np.random.randint(0, num_atoms, 5)
colors = np.full(num_atoms, 'black')
sizes = np.full(num_atoms, 3)
colors[highlight_points] = 'r'
sizes[highlight_points] = 30
scatter = displacements_plot.scatter(atom_coordinates[:,0], atom_coordinates[:,1], s=sizes, c=colors)

def animate_longitudinal_wave(i):
    old_coords[:,0] = displace_coordinates(atom_coordinates[:,0], 0) if i == 0 else new_coords[:,0]
    new_coords[:,0] = displace_coordinates(atom_coordinates[:,0], (i + 1) / num_frames)
    diffs[:] = new_coords - old_coords
    
    crossing_indices = np.where((new_coords[:,0] >= pressure_line_displacement - 4) & (new_coords[:,0] <= pressure_line_displacement + 4))
    crossing_diffs = diffs[crossing_indices]
    crossing_diff_magnitudes = np.linalg.norm(crossing_diffs, axis=1)
    total_pressure_values[i] = crossing_diffs[:,0].sum()

    scatter.set_offsets(new_coords)

    density_plot.cla()
    density_plot.hist(new_coords[:,0], bins=200)
    density_plot.set_title('Density')
    density_plot.set_xlabel('Displacement x (m)')
    density_plot.set_ylabel('Linear Density ($m^{-1}$)')
    density_plot.set_xlim([0, tube_length])
    density_plot.set_ylim([0, 50])

    colors = np.arctan2(diffs[:, 0], diffs[:, 1]) * np.linalg.norm(diffs, axis=1)
    colors /= colors.max()
    velocity_plot.cla()
    velocity_plot.quiver(new_coords[:, 0], new_coords[:, 1], diffs[:, 0], diffs[:, 1], color=cm.coolwarm(colors), pivot='mid')
    velocity_plot.axvline(x=pressure_line_displacement, linewidth=4, color='g', label='Measurement area')
    velocity_plot.set_title('Particle displacements with velocity')
    velocity_plot.set_xlabel('Displacement x (m)')
    velocity_plot.set_ylabel('Displacement y (m)')
    velocity_plot.set_xlim([0, tube_length])

    total_velocity_plot.cla()
    total_velocity_plot.plot(new_coords[:,0], diffs[:,0])
    total_velocity_plot.set_title('Total velocity at displacement')
    total_velocity_plot.set_xlabel('Displacement x (m)')
    total_velocity_plot.set_ylabel('Displacement y (m)')
    total_velocity_plot.set_xlim([0, tube_length])

    pressure_plot.cla()
    pressure_plot.plot(total_pressure_values[1:i], c='g')
    pressure_plot.set_title('Approx. pressure on green line above')
    pressure_plot.set_xlabel('Time')
    pressure_plot.set_ylabel('Sound Pressure Level (SPL)')
    pressure_plot.set_xlim([0, num_frames - 1])
    pressure_plot.yaxis.set_ticks([])

    fig.tight_layout()
    fig.subplots_adjust(top=0.95)

In [ ]:
create_animation(fig, plt, animate_longitudinal_wave, length_seconds=animation_length_seconds)

In [ ]:
def cartesian_to_polar(a):
    return np.array([np.arctan2(a[:,0], a[:,1]), np.linalg.norm(a, axis=1)]).T

def polar_to_cartesian(a):
    return np.array([a[:,1] * np.cos(a[:,0]), a[:,1] * np.sin(a[:,0])]).T

In [ ]:
np.random.seed(1234)

num_atoms = 6000
cyl_radius = 20
wave_length = cyl_radius / 8
wave_constant = 2 * np.pi / wave_length
amplitude = 0.10 * wave_length

pressure_line_displacements = [2 * cyl_radius / 6, 7 * cyl_radius / 8]
pressure_line_colors = ['orange', 'g']
total_pressure_values = [np.zeros(num_frames), np.zeros(num_frames)]

atom_coordinates = np.random.rand(num_atoms, 2) * 2 * cyl_radius - cyl_radius
polar_coords = cartesian_to_polar(atom_coordinates)
old_polar_coords = polar_coords.copy()
new_polar_coords = polar_coords.copy()
new_coords = atom_coordinates.copy()
old_coords = atom_coordinates.copy()
diffs = np.zeros(old_coords.shape)

fig, axes = plt.subplots(4, 1, figsize=(10, 24), gridspec_kw={'height_ratios':[3/8,3/8,1/8,1/8]})
fig.suptitle('Waves from Point Source', size=16)

displacements_plot = axes[0]
density_plot = axes[1]

displacements_plot.set_title('Particle displacements')
displacements_plot.set_xlabel('Displacement x (m)')
displacements_plot.set_ylabel('Displacement y (m)')
displacements_plot.set_xlim([-cyl_radius, cyl_radius])
displacements_plot.set_ylim([-cyl_radius, cyl_radius])

# highlight a few points on the scatter plot
highlight_points = np.random.randint(0, num_atoms, 5)
colors = np.full(num_atoms, 'black')
sizes = np.full(num_atoms, 3)
colors[highlight_points] = 'r'
sizes[highlight_points] = 30
scatter = displacements_plot.scatter(atom_coordinates[:,0], atom_coordinates[:,1], s=sizes, c=colors)

def animate_circular_wave(i):
    old_polar_coords[:,1] = displace_coordinates(polar_coords[:,1], 0) if i == 0 else new_polar_coords[:,1]
    new_polar_coords[:,1] = displace_coordinates(polar_coords[:,1], (i + 1) / num_frames)

    old_coords[:] = polar_to_cartesian(old_polar_coords)
    new_coords[:] = polar_to_cartesian(new_polar_coords)

    scatter.set_offsets(new_coords)

    diffs[:] = new_coords - old_coords

    colors = np.arctan2(diffs[:, 0], diffs[:, 1]) * np.linalg.norm(diffs, axis=1)
    colors /= colors.max()
    density_plot.cla()
    density_plot.quiver(new_coords[:, 0], new_coords[:, 1], diffs[:, 0], diffs[:, 1], color=cm.coolwarm(colors), pivot='mid')
    density_plot.set_title('Particle displacements with velocity')
    density_plot.set_xlabel('Displacement x (m)')
    density_plot.set_ylabel('Displacement y (m)')
    density_plot.set_xlim([-cyl_radius, cyl_radius])
    density_plot.set_ylim([-cyl_radius, cyl_radius])

    for j, pressure_line_displacement in enumerate(pressure_line_displacements):
        crossing_indices = np.where((new_coords[:,0] >= pressure_line_displacement - 8) & (new_coords[:,0] <= pressure_line_displacement + 8))
        crossing_diffs = diffs[crossing_indices]
        crossing_diff_magnitudes = np.linalg.norm(crossing_diffs, axis=1)
        total_pressure_values[j][i] = crossing_diffs[:,0].sum()

        density_plot.axvline(x=pressure_line_displacement, linewidth=4, color=pressure_line_colors[j], label='Measurement area')

        pressure_ax = axes[2 + j]
        pressure_ax.cla()
        pressure_ax.plot(total_pressure_values[j][1:i], c=pressure_line_colors[j])
        pressure_ax.set_title('Approx. pressure on ' + pressure_line_colors[j] + ' line above')
        pressure_ax.set_xlabel('Time')
        pressure_ax.set_ylabel('Sound Pressure Level (SPL)')
        pressure_ax.set_xlim([0, num_frames - 1])
        #pressure_ax.yaxis.set_ticks([])

    #fig.tight_layout()
    fig.subplots_adjust(top=0.95)

In [ ]:
create_animation(fig, plt, animate_circular_wave, length_seconds=animation_length_seconds)

# Psychoacoustic Basis of Sound

The ear actually takes advantage of the sinusoidal nature of sound by implementing a kind of discrete fourier transform with the cochlea.

## Critical Bands

![](https://cdn.britannica.com/s:700x450/04/14304-004-6C1B7EB1.jpg)
![](https://cdn.britannica.com/s:700x450/98/14298-004-99934987.jpg)

Zwicker and Feldtkeller (1955) played a narrowband noise signal containing all frequencies between 980 to 1020 Hz. The bandwidth of the signal was 40 Hz, and its band center was 1000 Hz. Then, keeping the band center at 1000 Hz, and keeping the total intensity constant, they gradually increased the bandwidth, spreading the same energy over a larger and larger frequency range.

Subjects reported that the loudness remained constant... but only up to a certain bandwidth, after which perceived loudness began to increase _even though there was no increase in total energy_. With the band center at 1 kHz, loudness began to increase when the bandwidth exceeded about 160 Hz.

It is hypothesized that areas of the basilar membrane respond together to selected frequency ranges - the critical bands.

**This is an important concept and is used for compression in MPEG/MP3 encoding.**

In [ ]:
from NoteSequence import render_samples_ipython
from generators import band_limited_noise
from conversion import SAMPLES_PER_SECOND

band_center_hz = 1_000
band_widths_hz = [32 + 2 ** i for i in range(3, 12)]
# The last band width is not really symmetric since anything below ~28hz will not contribute to the perceived power, but since this last band bump still ~close~ to doubles the number of new frequencies, its percieved loudness will still increase roughly proportionally.

band_limited_white_noise_signals = [band_limited_noise(band_center_hz - band_width_hz / 2, band_center_hz + band_width_hz / 2, SAMPLES_PER_SECOND * 2) for band_width_hz in band_widths_hz]
from scipy.signal import periodogram

for noise_signal in band_limited_white_noise_signals:
    f, Pxx_den = periodogram(noise_signal, SAMPLES_PER_SECOND)
    plt.semilogy(f, Pxx_den)
    plt.xlabel('frequency [Hz]')
    plt.xlim([band_center_hz - max(band_widths_hz) / 2 - 10, band_center_hz + max(band_widths_hz) / 2 + 10])
    plt.ylim([1e-9, 1e-6])
    plt.ylabel('PSD [V**2/Hz]')

# prevent clicks
ramp_samples = int(SAMPLES_PER_SECOND * 0.025)
for noise_signal in band_limited_white_noise_signals:
    noise_signal[:ramp_samples] *= np.linspace(0, 1, ramp_samples)
    noise_signal[-ramp_samples:] *= np.linspace(1, 0, ramp_samples)

render_samples_ipython(np.concatenate([noise_signal for noise_signal in band_limited_white_noise_signals]))

# Digital Signals and Sampling

### Nyquist Sampling Theorem (Aliasing)

Nyquist's theorem states that the apparant frequency $f_a$ of continuous frequency $f$ sampled at frequency $f_s$ can be expressed as

$f_a = f - \lfloor \frac{f}{f_s} + \frac{1}{2} \rfloor f_s$

To see why, consider this animation, where the cycle on the left is sampled and stored in the cycle on the right.

The apparent and actual frequncies are charted below.  You can see that the apparent frequency is limited by $\frac{+}{-}\frac{f_s}{2}$.  That is, it's limited to $1/2$ of the sampling frequency.

_Notice that the times when apparent frequency is 0 are the times when the orange spinning line on the right appears to change direction from clockwise (negative) to counterclockwise (positive) motion._

There is a meta-sampling thing happening here, too! Since the blue "continuous" line is sampled at the animation's frame rate 40 FPS, when the blue line spins at $\frac{\text{FPS}}{2} = 20 Hz$ we see a distinct change in direction.  (This crossing point is shown in the second chart.)

In [ ]:
def apparent_frequency(f, f_s):
    return (f - np.floor(f / f_s + 0.5) * f_s)

In [ ]:
import sys
sys.path.append('../')
sys.path.append('../../')

from ipython_animation import create_animation
import matplotlib.pyplot as plt

%matplotlib inline

animation_length_seconds = 16
frames_per_second = 40
num_frames = animation_length_seconds * frames_per_second

continuous_coordinates = np.zeros((2, 2))
sampled_coordinates = np.zeros((2, 2))

continuous_coordinates[0,0] = -1
sampled_coordinates[0,0] = 1

fig = plt.figure(figsize=(10, 16))
plt.subplot(311)
continuous_line, = plt.plot(continuous_coordinates[0,:], continuous_coordinates[1,:], linewidth=4, label='Continuous signal')
sampled_line, = plt.plot(sampled_coordinates[0,:], sampled_coordinates[1,:], linewidth=4, label='Sampled signal')
plt.xlim(-2.1, 2.1)
plt.ylim(-1, 1)
plt.legend(loc='upper center')

f_s = 8
sample_frames = int(frames_per_second / f_s)
f_velocity = 1.5 / frames_per_second

plt.subplot(312)
plt.title('Apparent Frequency')

t_range = np.arange(num_frames, dtype='int')
f_history = np.ma.zeros(num_frames)
f_a_history = np.ma.zeros(num_frames)

f_line, = plt.plot(f_history, label='$f$')
f_a_line, = plt.plot(f_a_history, label='$f_a$')
plt.hlines(y=f_s / 2, xmin=0, xmax=num_frames, label='$\\frac{1}{2}$ sample rate ($f_s$)', color='r')
plt.hlines(y=frames_per_second / 2, xmin=0, xmax=num_frames, label='$\\frac{1}{2}$ animation FPS')
plt.legend(loc='upper left')

plt.ylim(-f_s / 2 - 0.1, f_velocity * num_frames + 0.1)

plt.subplot(313)
plt.title('Position')
pos_history = np.full(num_frames, np.nan)
sample_pos_history = np.full(num_frames, np.nan)
pos_line, = plt.plot(pos_history, label='actual')
sample_pos_line, = plt.plot(sample_pos_history, label='sample', marker='o', linewidth=3, markevery=sample_frames)
plt.xlim(0, num_frames)
plt.ylim(-1.1, 1.1)
plt.legend(loc='upper left')

theta_mutable = [0]

def animate_nyquist(i):
    f = i * f_velocity
    f_history[i] = f
    f_a_history[i] = apparent_frequency(f, f_s)
    theta_mutable[0] += 2 * np.pi * f * (1 / frames_per_second)
    theta = theta_mutable[0]
    coordinates = [np.cos(theta), np.sin(theta)]
    continuous_coordinates[:,1] = continuous_coordinates[:,0] + coordinates
    pos_history[i] = coordinates[0]

    if i % sample_frames == 0:
        sampled_coordinates[:,1] = sampled_coordinates[:,0] + coordinates
        if i == 0:
            sample_pos_history[i] = pos_history[i]
        else:
            sample_pos_history[i - sample_frames:i + 1] = np.linspace(pos_history[i - sample_frames], pos_history[i], sample_frames + 1)
    continuous_line.set_data([continuous_coordinates[0,:], continuous_coordinates[1,:]])
    sampled_line.set_data([sampled_coordinates[0,:], sampled_coordinates[1,:]])
    f_line.set_ydata(np.ma.masked_where(t_range > i, f_history))
    f_a_line.set_ydata(np.ma.masked_where(t_range > i, f_a_history))
    
    pos_line.set_ydata(pos_history)
    sample_pos_line.set_ydata(sample_pos_history)

In [ ]:
create_animation(fig, plt, animate_nyquist, length_seconds=animation_length_seconds, frames_per_second=frames_per_second)

### Consequences of aliasing

Aliasing can cause distortion, since a low sample rate will not be able to represent high-frequency content.  For example, consider this synthesized violin tone with the following harmonics (from p17):

In [ ]:
from NoteSequence import render_notes_ipython

fundamental = 440
violin_harmonics = [fundamental * i for i in range(1, 14)]
violin_notes = [[(harmonic, 3, 1.0/(i + 1))] for i, harmonic in enumerate(violin_harmonics)]

render_notes_ipython(violin_notes)

In [ ]:
sample_rate = 10_000
violin_harmonics_aliased = [np.abs(apparent_frequency(harmonic, sample_rate)) for harmonic in violin_harmonics]
print(violin_harmonics_aliased)
violin_notes_aliased = [[(harmonic, 3, 1.0/(i + 1))] for i, harmonic in enumerate(violin_harmonics_aliased)]

render_notes_ipython(violin_notes_aliased)

These frequencies are no longer integer multiples of the fundamental, and thus no longer harmonics.

## Constructing a Frequency Detector

The fundamental insight of the Fourier transform is:

_**The more positive the product signal is, the closer the source signals are to being identical.**_

_**The more mixed positive and negative the product signal is, the less identical are the source signals.**_

We can use this principle to test for a probe signal at every frequency multiple in a certain range:

In [ ]:
from ipython_animation import create_animation, DEFAULT_FPS
import matplotlib.cm as cm

def create_dst_animation(test_signal, animation_length_seconds):
    N = len(test_signal)
    frequency_changing = 0.05
    num_frames = animation_length_seconds * DEFAULT_FPS

    fig = plt.figure(figsize=(10, 10))
    probe_signal = np.sin(2 * np.pi * frequency_changing * np.linspace(0, 1, N))
    product_plot = plt.subplot(311)
    product_plot.set_title('Product of two sinusoid signals\n(More positive product == closer to identical)', size=14)
    test_line, = product_plot.plot(test_signal, linewidth=3, label='Test signal')
    probe_line, = product_plot.plot(probe_signal, linewidth=3, label='Probe signal')
    product_plot.legend(loc='upper right')
    product_signal_plot = plt.subplot(312)
    product_line, = product_signal_plot.plot(test_signal * probe_signal, linewidth=3)
    product_signal_plot.set_ylim(-1.1, 1.1)
    product_signal_plot.set_title('Product signal')

    product_sum_plot = plt.subplot(313)
    product_sums = np.array([np.arange(num_frames), np.zeros(num_frames)]).T
    product_sum_colors = ['w'] * num_frames
    product_sum_line = product_sum_plot.scatter(product_sums[:,0], product_sums[:,1], color=product_sum_colors, s=12)

    product_sum_plot.set_ylim(-20, (test_signal ** 2).sum() + 5)
    product_sum_plot.set_title('Product sum')
    plt.tight_layout()

    def animate(i):
        probe_signal = np.sin(2 * np.pi * i * frequency_changing * np.linspace(0, 1, N))
        probe_line.set_ydata(probe_signal)
        product_signal = test_signal * probe_signal
        product_line.set_ydata(product_signal)
        product_sums[i,1] = product_signal.sum()
        color = cm.coolwarm(product_sums[i,1] / (test_signal ** 2).sum())
        product_sum_colors[i] = color
        product_line.set_color(color)
        product_sum_plot.set_title('Product sum = %0.01f' % product_signal.sum(), color=color)
        product_sum_line.set_offsets(product_sums)
        product_sum_line.set_color(product_sum_colors)

    return create_animation(fig, plt, animate, length_seconds=animation_length_seconds)

In [ ]:
N = 100
frequency_stationary = 4
test_signal = np.sin(2 * np.pi * frequency_stationary * np.linspace(0, 1, N))
animation_length_seconds = 6

create_dst_animation(test_signal, animation_length_seconds)

### _Side note:_ sinusoids at the same frequency

An important property of sinusoids at a particular frequency is that they are _closed_ with respect to addition. In other words, if you take a sinusoid, make many copies of it, scale them all by different gains, delay them all by different time intervals, and add them up, you always get a sinusoid at the same original frequency.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

num_sinusoids = 10
amplitudes = np.random.rand(num_sinusoids) * 10
phase_offsets = np.random.rand(num_sinusoids) * 2 * np.pi

sinusoids = np.array([amplitudes[i] * np.sin(np.linspace(-2 * np.pi, 2 * np.pi, 100) + phase_offsets[i]) for i in range(num_sinusoids)])
sum_of_sinusoids = np.sum(sinusoids, axis=0)

plt.title('Sum of random scaled and delayed sinusoids\nwith the same frequency')
plt.plot(sinusoids.T)
plt.plot(sum_of_sinusoids, linewidth=4, label='Sum of sinusoids')
_ = plt.legend(loc='lower right')

This animation basically _is_ the Discrete Fourier Transform! Well, it's close. We can call this the **Discrete Sine Transform (DST)**, since it tells us how much of a sine wave there is at any given frequency.

BUT! This solution doesn't work well when we give it a signal that is out of phase with the sine wave. For example, let's try a simple sine wave with frequency $f = 2$, and see what happens as we change the phase $\phi$ slowly from $0$ to $2\pi$. In addition to our **DST** algorithm, let's also add a chart with a _**cosine**_ test signal (which we'll call the **Discrete Cosine Transform**:

Neither of these functions is capable of detecting signals exactly out of phase with their "probe" signal - the DCT will detect no energy if fed a sine signal and the DST will not be able to detect a cosine signal.

In [ ]:
def discrete_sine_transform(x, k):
    N = len(x)
    return (x * np.sin(k * 2 * np.pi * np.linspace(0, 1, N))).sum() / N

def discrete_cosine_transform(x, k):
    N = len(x)
    return (x * np.cos(k * 2 * np.pi * np.linspace(0, 1, N))).sum() / N

In [ ]:
frequency = 2

fig, plots = plt.subplots(3, 1, figsize=(8, 10))
signal_plot = plots[0]
signal_line, = signal_plot.plot(np.zeros(N))
signal_plot.set_xlim(0, N - 1)
signal_plot.set_ylim(-1, 1)

dst_plot = plots[1]
dct_plot = plots[2]
ks = np.arange(-3, 4)
dct_scatter = dct_plot.scatter(ks, np.zeros(ks.size), c='black')
dst_scatter = dst_plot.scatter(ks, np.zeros(ks.size), c='black')

animation_length_seconds = 4
num_frames = animation_length_seconds * DEFAULT_FPS

def animate_dct_dst(i):
    phase = 2 * np.pi * i / num_frames
    x = np.sin(2 * np.pi * frequency * np.linspace(0, 1, N) + phase)
    signal_line.set_ydata(x)
    signal_plot.set_title('Test signal with $f = %i$ and $\phi = %0.3f\\pi$' % (frequency, i / num_frames))
    
    for label in ['DST', 'DCT']:
        plot = dct_plot if label == 'DCT' else dst_plot
        scatter = dct_scatter if label == 'DCT' else dst_scatter
        partial_dft_function = discrete_cosine_transform if label == 'DCT' else discrete_sine_transform
        transform = np.array([partial_dft_function(x, k) for k in ks])
        offsets = np.array([ks, transform]).T
        scatter.set_offsets(offsets)
        plot.set_xlim(-3, 3)
        plot.set_ylim(-1, 1)
        plot.set_xlabel('Frequency')
        plot.set_ylabel('Magnitude')
        plot.set_title(label)
        plt.tight_layout()

In [ ]:
create_animation(fig, plt, animate_dct_dst, length_seconds=animation_length_seconds)

Notice that when $\phi = 0$ and the signal is a pure sine, the DST is at maximum energy and the DCT is at 0 energy, and vice versa for $\phi = \frac{\pi}{4}$, when the signal is a pure cosine.

### Finding a Sinusoid from a Sum of Sine and Cosine

We can combine the DST and the DCT two to detect the amplitude and phase of sinusoids with arbitrary phase.

For any frequency $\theta$ and phase $\phi$,

$A\sin(\theta + \phi) = a\cos\theta+b\sin\theta$

for suitable choices of $A$, $\phi$, $a$, and $b$.

The discrete cosine transform gives us the _even_ components of the spectrum ($a_k$), and the discrete sine transform gives us the _odd_ components of the spectrum ($b_k$).

By the Pythagorian theorem, the amplitude of each component is

$A(k) = \sqrt{a^2_k + b^2_k}$.

The phase of each component can be found with

$\phi(k) = \tan^{-1}\frac{b_k}{a_k}$.

In [ ]:
# I am calling this "inelegant DFT" because it combines the DST and the DCT to
# probe for sinusoids of arbitrary phase, even though there is a simpler way.
def inelegant_dft(x, k):
    dct = discrete_cosine_transform(x, k)
    dst = discrete_sine_transform(x, k)
    return np.sqrt(dct ** 2 + dst ** 2), np.arctan2(-dst, dct)

In [ ]:
fig, plots = plt.subplots(3, 1, figsize=(8, 10))
signal_plot = plots[0]
signal_line, = signal_plot.plot(np.zeros(N))
signal_plot.set_xlim(0, N - 1)
signal_plot.set_ylim(-1, 1)

mag_plot = plots[1]
mag_plot.set_xlim(-3, 3)
mag_plot.set_ylim(-1, 1)
mag_plot.set_xlabel('Frequency')
mag_plot.set_ylabel('Magnitude')
mag_plot.set_title('Magnitude spectrum from combined DCT and DST')

phase_plot = plots[2]
phase_plot.set_title('Phase spectrum from combined DCT and DST')
phase_plot.set_xlim(-3, 3)
phase_plot.set_ylim(-2 * np.pi, 2 * np.pi)

frequency = 2
ks = np.arange(-3, 4)
mag_scatter = mag_plot.scatter(ks, np.zeros(ks.size), c='black')
phase_scatter = phase_plot.scatter(ks, np.zeros(ks.size), c='green')

def animate_inelegant_dft(i):
    phase = 2 * np.pi * i / num_frames
    x = np.sin(2 * np.pi * frequency * np.linspace(0, 1, N) + phase)
    signal_line.set_ydata(x)
    signal_plot.set_title('Test signal with $f = %i$ and $\phi = %0.3f\\pi$' % (frequency, i / num_frames))
    transform = np.array([inelegant_dft(x, k) for k in ks])
    mag_scatter.set_offsets(np.array([ks, transform[:,0]]).T)
    phase_scatter.set_offsets(np.array([ks, transform[:,1]]).T)
    plt.tight_layout()

In [ ]:
create_animation(fig, plt, animate_inelegant_dft, length_seconds=animation_length_seconds)

## Complex Harmonic Motion

_We can simplify this greatly by using complex numbers!_

The orthogonal projection of the real and imaginary components of a complex phasor ($e^{i\theta}$) shows that complex phasors in fact contain both a $\sin$ and $\cos$ component.

In [ ]:
from ipython_animation import DEFAULT_FPS

from matplotlib.patches import Circle
from mpl_toolkits.mplot3d import Axes3D
import mpl_toolkits.mplot3d.art3d as art3d

animation_length_seconds = 6
num_frames = DEFAULT_FPS * animation_length_seconds

fig = plt.figure(figsize=(12, 12))
ax = plt.gca(projection='3d')

fig.suptitle('Complex sinusoid projected on real and imaginary axis', size=16)
plt.xlabel('Time')
plt.ylabel('Real projection ($\cos$)')
ax.set_zlabel('Imaginary projection ($\sin$)')

radius = 1
center = (0.0, 0.0)
# Draw a circle on the x=0 'wall'
p = Circle(center, radius, color='black', alpha=0.2)
ax.add_patch(p)
art3d.pathpatch_2d_to_3d(p, z=0, zdir='x')

xlim = [0, 1]
ylim = [-2, 2]
zlim = [-2, 2]
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_zlim(zlim)
projection_line_x, = ax.plot([xlim[0], xlim[0]], center, [center[0] + radius, zlim[1]], linewidth=3, label='Real')
projection_line_y, = ax.plot([xlim[0], xlim[0]], [center[0] + radius, 1], center, linewidth=3, label='Imaginary')
radius_line, = ax.plot([xlim[0], xlim[0]], [ylim[0], ylim[0]], [center[0], center[0] + radius], c='r', linewidth=3, label='Complex')
projection_y_scatter = ax.scatter(-1, -1, -1)
projection_z_scatter = ax.scatter(-1, -1, -1)
ax.legend()

ys = []
zs = []

theta_mutable = [0]
angular_velocity = 4 * np.pi / num_frames

def animate_complex_projection(i):
    theta_mutable[0] += angular_velocity
    theta = theta_mutable[0]
    phasor = np.e ** (1j * theta)
    y = center[0] + phasor.real
    z = center[1] + phasor.imag

    radius_line.set_xdata([xlim[0], xlim[0]])
    radius_line.set_ydata([center[0], y])
    radius_line.set_3d_properties([center[1], z])

    projection_line_x.set_xdata([xlim[0], xlim[0]])
    projection_line_x.set_ydata([y, y])
    projection_line_x.set_3d_properties([zlim[0], z])

    projection_line_y.set_xdata([xlim[0], xlim[0]])
    projection_line_y.set_ydata([y, ylim[1]])
    projection_line_y.set_3d_properties([z, z])

    if not (i == 0 and len(zs) > 0):
        # animates at frame 0 twice for some reason...
        ys.insert(0, y)
        zs.insert(0, z)
        
    projection_y_scatter._offsets3d = np.linspace(xlim[0], (i + 1) / num_frames, len(ys)), np.array(ys), np.full(len(ys), zlim[0])
    projection_z_scatter._offsets3d = np.linspace(xlim[0], (i + 1) / num_frames, len(zs)), np.full(len(zs), zlim[1]), np.array(zs)

In [ ]:
create_animation(fig, plt, animate_complex_projection, length_seconds=animation_length_seconds)

Here's a simple 3D plot showing the motion of a phasor through complex-coordinate space over time:

In [ ]:
angles = np.linspace(0, 8 * np.pi, 200)
phasors = np.e **(1j * angles)

fig = plt.figure(figsize=(12, 12))
ax = plt.gca(projection='3d')

ax.set_xlim(angles.min(), angles.max())
ax.set_ylim(-2, 2)
ax.set_zlim(-2, 2)
ax.set_xlabel('$\\theta$')
ax.set_ylabel('Real')
ax.set_zlabel('Imaginary')

ax.plot(angles, phasors.real, np.full(phasors.size, -2), linewidth=4, alpha=0.5, label='Real')
ax.plot(angles, np.full(phasors.size, 2), phasors.imag,  linewidth=4, alpha=0.5, label='Imaginary')
ax.plot(angles, phasors.real, phasors.imag, linewidth=4, label='Complex', c='r')
ax.set_title('Complex sinusoid', y=1.04, size=14)
_ = ax.legend()

## Complex Numbers

Define $i \triangleq \sqrt{-1}$.

Then,

$i^2 = -1\\
i^3 = (i^2)\cdot i = -i\\
i^4 = (i^3)\cdot i = -i\cdot i = -i^2 = 1$

In [ ]:
def plot_powers_of_i(powers_of_i, title='Plotting $i^n$ on the complex plane', show_labels=True):
    plt.figure(figsize=(8, 8))
    plt.title(title)
    plt.xlabel('Real part')
    plt.ylabel('Imaginary part')
    plt.xlim(-1.5, 1.5)
    plt.ylim(-1.5, 1.5)
    plt.grid(True)
    plt.scatter(powers_of_i.real, powers_of_i.imag, s=100)

    if show_labels:
        for power in np.arange(1, len(powers_of_i) + 1):
            z = 1j ** power
            plt.annotate('$i^%i$' % power, (z.real + 0.04, z.imag + 0.04), size=20)

We can plot any complex number $z = x + iy$ in a plane as an ordered pair $(x, y)$:

In [ ]:
plot_powers_of_i(1j ** np.arange(1, 5))

We can generalize this to non-integer exponentials to create a circle!

In [ ]:
plot_powers_of_i(1j ** np.linspace(0, 4, 100), show_labels=False)

But how to specify a specific radian angle $\theta$ on the unit circle?

It turns out we can use $e^{i\theta}$ (because magic):

In [ ]:
angles = np.linspace(0, 2 * np.pi, 100)
plot_powers_of_i(np.e ** (1j * angles), title='Plotting $e^{i\\theta}$ on the complex plane', show_labels=False)

## DFT in equation form

$X(k) = \frac{1}{N}\sum_\limits{n=0}^\limits{N-1}x(n)e^{-ik\omega n / N}$


The complex return values of $X(k)$ contain both the amplitude spectrum and the phase spectrum.

In [ ]:
def dft(x, k):
    N = len(x)
    return (x * np.exp(-1j * k * 2 * np.pi * np.linspace(0, 1, N))).sum() / N

## Analyzing Real-World Signals

In [ ]:
def get_positive_frequencies(X):
    return X[-1:-1-(X.size // 2):-1]

def get_magnitude_spectrum(x):
    return np.abs(get_positive_frequencies(dft(x))) * 2

In [ ]:
x = np.sin(2 * np.pi * 4 * np.linspace(0, 1, 100))
fig = plt.figure(figsize=(9, 6))
x_plot = plt.subplot(211)
x_plot.plot(x)
x_plot.set_title('Test signal - pure $\sin$ with $f = 4$')
x_plot.set_xlim(0, len(x) - 1)
spectrum_plot = plt.subplot(212)
X = get_magnitude_spectrum(x)
spectrum_plot.plot(X, c='r')
spectrum_plot.set_title('Spectrum of pure $\sin$ test signal')
spectrum_plot.set_xlabel('Positive increasing frequencies')
spectrum_plot.set_ylabel('Magnitude')
spectrum_plot.set_xlim(0, len(X) - 1)
plt.tight_layout()

### Windowing

Windowing helps reduce the _leakage_ problem, where discontinuities at the edge of the analysis window introduce noise throughout the spectrum.  The energy that should be in one spectral harmonic spreads into adjacent harmonics because the DFT assums it is given _one period of an infinitely repeating periodic function_ at the fundamental analysis frequency.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(10, 8))

signal_plot = plt.subplot(221)
signal_plot.set_title('Sine before windowing')
signal_plot.set_xlabel('angle (radians)')
signal_plot.set_ylabel('sin(x)')

spectrum_plot = plt.subplot(222)
spectrum_plot.set_title('Spectrum before windowing')

signal_plot_windowed = plt.subplot(223)
signal_plot_windowed.set_title('Sine after windowing')
signal_plot_windowed.set_xlabel('angle (radians)')
signal_plot_windowed.set_ylabel('sin(x) (Windowed)')

spectrum_plot_windowed = plt.subplot(224)
spectrum_plot_windowed.set_title('Spectrum after windowing\n(using Blackman window)')

N = 64
x = np.linspace(0, 2 * np.pi, N) # multiple of pi will be exactly periodic over window length
signal_line, = signal_plot.plot(x, np.zeros(N), label='Angle (radians)')
X_line, = spectrum_plot.plot(np.zeros(N // 2), color='r')
signal_windowed_line, = signal_plot_windowed.plot(x, np.zeros(N), label='Angle (radians)')
X_line_windowed, = spectrum_plot_windowed.plot(np.zeros(N // 2), color='r')

signal_plot.axis([0, x.max(), -1, 1])
signal_plot_windowed.axis([0, x.max(), -1, 1])
spectrum_plot.axis([0, N // 2 - 1, 0, 1.1])
spectrum_plot_windowed.axis([0, N // 2 - 1, 0, 1.1])
plt.tight_layout()

window = np.blackman(N)
def animate_windowing(i):
    freq = (i + 100) / 25
    y = np.sin(freq * x)
    X = get_magnitude_spectrum(y)
    signal_line.set_ydata(y)
    X_line.set_ydata(X)
    spectrum_plot.legend(handles=[X_line], labels=['Freq (radians): {0:.2f}'.format(freq)])

    y_windowed = y * window
    signal_windowed_line.set_ydata(y_windowed)
    X_windowed = get_magnitude_spectrum(y_windowed)
    X_line_windowed.set_ydata(X_windowed)
    spectrum_plot_windowed.legend(handles=[X_line_windowed], labels=['Freq (radians): {0:.3f}'.format(freq)])

In [ ]:
create_animation(fig, plt, animate_windowing, length_seconds=4, default_mode='reflect')